In [1]:
import numpy as np
import numba
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt

In [2]:
@numba.njit
def one_hotify(t, size):
    t_oh = np.zeros((t.shape[0], size))
    for idx, i in enumerate(t):
        t_oh[idx][i] = 1
        
    return t_oh

# Network Blocks and their Derivatives

In [3]:
@numba.njit
def d_softmax(x):
    return softmax(x) * (1 - softmax(x))


@numba.njit
def d_relu(x):
    return (x > 0).astype(np.float32)

@numba.njit
def clip_gradients(gradients, max_norm=1.0):
    if max_norm is None:
        return gradients
    
    norm = np.linalg.norm(gradients, ord=2)
    clip_coeff = max_norm / (norm + 1e-6)
    if clip_coeff < max_norm:
        clipped = gradients * clip_coeff
    else:
        clipped = gradients
        
    return clipped


class Layer(ABC):
    @abstractmethod
    def back_fn(self, prev, x):
        pass

    def update_weights(self, dW, db, lr=1e-3, clip_val=1.0):
        pass


class LinearLayer(Layer):
    def __init__(self, in_shape, out_shape):
        self.in_shape = in_shape
        self.out_shape = out_shape
        limit = np.sqrt(6 / (in_shape + out_shape))
        self.weights = np.random.uniform(-limit, limit, (in_shape, out_shape))
        self.bias = np.zeros((1, out_shape))

    def back_fn(self, prev, x):
        dW = np.dot(x.T, prev)  # Assuming x and prev are 2D matrices

        # Gradient of the loss with respect to the biases
        db = np.sum(prev, axis=0, keepdims=True)  # Summing over all samples if in batch

        # Gradient of the loss with respect to the input of this layer
        dx = np.dot(prev, self.weights.T)

        return dx, dW, db

    def update_weights(self, dW, db, lr=1e-3, clip_val=1.0):
        self.weights -= (clip_gradients(dW, clip_val) * lr)
        self.bias -= (clip_gradients(db, clip_val) * lr)

    def __call__(self, x):
        return x @ self.weights + self.bias

    def __str__(self):
        return f"LinearLayer(in_shape={self.in_shape}, out_shape={self.out_shape})"


class ReluLayer(Layer):
    def back_fn(self, prev, x):
        drelu = d_relu(x)
        dx = prev * drelu
        return dx, None, None

    def __call__(self, x):
        return np.where(x > 0, x, 0)

    def __str__(self):
        return f"ReluLayer()"


def softmax(x):
    # Shift x for numerical stability by subtracting its max value in each row
    shift_x = x - np.max(x, axis=1, keepdims=True)
    exp_shift_x = np.exp(shift_x)
    softmax = exp_shift_x / np.sum(exp_shift_x, axis=1, keepdims=True)
    return softmax


class SoftmaxLayer:
    def __call__(self, x):
        return softmax(x)
    
    def __str__(self):
        return "SoftmaxLayer()"


def log_softmax(x):
    x_off = x - np.max(x)
    return x_off - np.log(np.sum(np.exp(x_off), axis=-1, keepdims=True))


class ComposedNetwork:
    def __init__(self, *layers):
        self.layers = layers
        self.inputs = []

    def backward(self, dx, lr=1e-3, clip_val=1.0):
        """dx should be the gradient of the cost function."""
        for layer, input in zip(reversed(self.layers), reversed(self.inputs)):
            dx, dW, db = layer.back_fn(dx, input)
            layer.update_weights(dW, db, lr)
            
        self.inputs.clear()

    def __call__(self, x):
        self.inputs.clear()
        for layer in self.layers:
            self.inputs.append(x)
            x = layer(x)

        return x

    def __str__(self):
        compose = "ComposedNetwork(\n"
        for layer in self.layers:
            compose += "\t"
            compose += str(layer)
            compose += ",\n"
        compose += ")"

        return compose

In [4]:
network = ComposedNetwork(
    *[
        LinearLayer(768, 128),
        ReluLayer(),
        LinearLayer(128, 128),
        ReluLayer(),
        LinearLayer(128, 128),
        ReluLayer(),
        LinearLayer(128, 10),
    ]
)

print(network)

ComposedNetwork(
	LinearLayer(in_shape=768, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=10),
)


# Loss Functions

In [5]:
@numba.njit
def nll_loss(pred, target):
    s = 0
    idx = 0
    for t in target:
        s += pred[idx][t]
        idx += 1
    
    return -s / idx
    

def cross_entropy(y_pred, y_true):
    # Avoid division by zero
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    
    # Compute the loss for each sample and class
    loss = -np.sum(y_true * log_softmax(y_pred)) / y_pred.shape[0]
    return loss

def d_cross_entropy(y_pred, y_true):
    # Compute the gradient
    grad = y_pred - y_true
    return grad


def mse_loss(pred, target):
    return np.mean((target - pred) ** 2)

def d_mse(pred, target):
    n = pred.shape[0]  # Assuming pred and target are numpy arrays of the same shape
    return (2/n) * (pred - target)

# The Network

Let's try and overfit to our little dummy dataset.

In [6]:
network = ComposedNetwork(
    *[
        LinearLayer(784, 128),
        ReluLayer(),
        LinearLayer(128, 128),
        ReluLayer(),
        LinearLayer(128, 128),
        ReluLayer(),
        LinearLayer(128, 10),
    ]
)

print(network)

ComposedNetwork(
	LinearLayer(in_shape=784, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=128),
	ReluLayer(),
	LinearLayer(in_shape=128, out_shape=10),
)


In [7]:
import torch
import torch.utils.data
import torchvision
import torchvision.transforms as tr

mnist = torchvision.datasets.MNIST(
    "./data",
    transform=tr.Compose(
        [tr.ToTensor(), tr.Normalize(0.5, 0.5), lambda x: x.reshape(784)]
    ),
    download=True
)

dataloader = torch.utils.data.DataLoader(mnist, 128)

In [8]:
from tqdm import tqdm

n_epoch = 100
lr = 1e-3

loss_chart = []
acc_chart = []

i = 0

for epoch in range(n_epoch):
    for inputs, labels in tqdm(dataloader):
        labels = labels.numpy()
        inputs = inputs.numpy()
        labels = one_hotify(labels, 10)

        out = network(inputs)

        loss = cross_entropy(out, labels)
        acc = (
            np.argmax(softmax(out), axis=1) == np.argmax(labels, axis=1)
        ).mean() * 100

        if np.isnan(loss):
            break

        d_loss = d_cross_entropy(out, labels)

        if i % 100 == 99:
            i *= 0.5

        network.backward(d_loss, lr)

        loss_chart.append(loss)
        acc_chart.append(acc)
        i += 1

    print(loss, acc)
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    ax[0].plot(loss_chart)
    ax[0].set_title("Loss")
    
    ax[1].plot(acc_chart)
    ax[1].set_title("Accuracy")
    
    plt.show()

  0%|          | 0/469 [00:00<?, ?it/s]